In [2]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

search_img = cv2.imread('search.png')
template_img = cv2.imread('template.png')
search_lab = cv2.cvtColor(search_img, cv2.COLOR_BGR2Lab)
template_lab = cv2.cvtColor(template_img, cv2.COLOR_BGR2Lab)

template_height, template_width = template_img.shape[:2]
half_width = template_width // 2
half_height = template_height // 2

scores = []
positions = []

for y in range(half_height, search_img.shape[0] - half_height):
    for x in range(half_width, search_img.shape[1] - half_width):
        # Extract the region of interest from the search image and resize it to the template size
        roi = search_lab[y - half_height:y + half_height, x - half_width:x + half_width]
        roi_resized = cv2.resize(roi, (template_width, template_height))
        
        # Normalize the template and the resized region of interest
        normalized_template = (template_lab - np.mean(template_lab)) / np.std(template_lab)
        normalized_roi = (roi_resized - np.mean(roi_resized)) / np.std(roi_resized)
        ncc_metric = np.sum(normalized_template * normalized_roi)

        scores.append(ncc_metric)
        positions.append((x, y))

# Sort the scores and positions in descending order
sorted_indices = np.argsort(scores)[::-1]
sorted_scores = np.array(scores)[sorted_indices]
sorted_positions = np.array(positions)[sorted_indices]

# Define the indices for specific matches to visualize
indices_to_visualize = [0, 1, 4, 9, 99, 499]

# Display the patches for specific matches
for i in indices_to_visualize:
    x, y = sorted_positions[i]
    patch = search_img[y - half_height:y + half_height, x - half_width:x + half_width]
    cv2.imshow(f'Patch {i+1}', patch)

cv2.waitKey(0)
cv2.destroyAllWindows()